### **Libraries**

In [1]:
import os
import openai
from openai import OpenAI

In [2]:
import pathlib
import textwrap

import openai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # OpenAI API key

#### **Define a client**

In [4]:
# define an instance of OpenAI model
client = OpenAI(api_key=OPENAI_API_KEY)

#### **Audio transcription**

In [5]:
audio_file= open("longer_schedule.mp3", "rb")

In [28]:
speech_to_text = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
)

In [7]:
to_markdown(speech_to_text.text)

> On the 2nd of July, I will be having a masterclass with Julian. While on the 7th, I will be having a synchronized learning with Sik Og. On the 14th of July, by 7pm WET, I will be having another synchronized learning. On the 21st of July, I will be having a synchronized learning between the hours of 7pm to 9pm with Sik Og Analytics. While on the 21st, I will be doing a final presentation with Sik Og Analytics between the hours of 7pm to 9pm.

### **Audio to file**

### **Function calling**

In [22]:
system_prompt = "You are an assistant that creates a well crafted schedule from text. Extract all events related data from the provided text."

In [43]:
schedule_tool = [
    {
        'type': 'function',
        'function': {
            'name': 'schedule_events',
            'description': 'Extract all events related data from the provided text. If there are multiple events extract all events',
            'parameters': {
                'type': 'object',
                'properties': {
                    'date': {
                        'type': 'list',
                        'description': 'The date of the event in the format YYYY-MM-DD'
                    },
                    'start_time': {
                        'type': 'list',
                        'description': 'The time of the event starts'
                    },
                    'end_time': {
                        'type': 'list',
                        'description': 'The time of the event ends'
                    },
                    'event': {
                        'type': 'list',
                        'description': 'The name of the event'
                    },
                    'location': {
                        'type': 'list',
                        'description': 'The location of the event'
                    },
                    'description': {
                        'type': 'list',
                        'description': 'A description of the event'
                    },
                    'reminder': {
                        'type': 'list',
                        'description': 'The time before the event to send a reminder'
                    }
                }
            },
            'required': ['date', 'start_time', 'end_time', 'event'],
        },
        'instruction': system_prompt
    }
]

#### **Calendar file creation**

In [44]:
query = """
On the 2nd of July, I will be having a masterclass with Julian between the hours of 3:00 PM and 4:00 PM WAT. While on the 7th, \
I will be having a synchronized learning with Sik Og. On the 14th of July, by 7pm WET, \
I will be having another synchronized learning. On the 21st of July, I will be having a synchronized \
learning between the hours of 7pm to 9pm with Sik Og Analytics. While on the 21st, I will be doing a final \
presentation with Sik Og Analytics between the hours of 7pm to 9pm.
"""

In [45]:
user_prompt = f"From the text {query} create a calendar-like structure."

calendar_file = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'user', 'content': user_prompt}
    ],
    tools=schedule_tool,
    tool_choice='required',
    temperature=0.3
)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for function 'schedule_events': In context=('properties', 'date'), array schema missing items", 'type': 'invalid_request_error', 'param': 'tools[0].function.parameters', 'code': 'invalid_function_parameters'}}

: 

In [41]:
print(calendar_file)

ChatCompletion(id='chatcmpl-9hidsXRokt3V0MA5U0xgcwI5ny8G2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Df7AHL0oi1iB798sLnBqIhdQ', function=Function(arguments='{"date": ["2023-07-02"], "start_time": ["15:00"], "end_time": ["16:00"], "event": ["Masterclass with Julian"], "location": [], "description": [], "reminder": []}', name='schedule_events'), type='function'), ChatCompletionMessageToolCall(id='call_xARovoGP270e8GuJf11QLBya', function=Function(arguments='{"date": ["2023-07-07"], "start_time": ["00:00"], "end_time": ["23:59"], "event": ["Synchronized learning with Sik Og"], "location": [], "description": [], "reminder": []}', name='schedule_events'), type='function'), ChatCompletionMessageToolCall(id='call_FNnhr9Hn3rSLARfIKTuoBtWt', function=Function(arguments='{"date": ["2023-07-14"], "start_time": ["19:00"], "end_time": ["20:00"], "e

In [42]:
if(calendar_file.choices[0].message.tool_calls):
  #If tool call was invoked extract the name of the specific function used
  function_name = calendar_file.choices[0].message.tool_calls[0].function.name
  #Print the name of the function called
  print("called the function: " + function_name)
  # Extract response to user query
  result = calendar_file.choices[0].message.tool_calls[0].function.arguments
else:
  print("didn't call a function")
  result = calendar_file.choices[0].message.content
  
to_markdown(result)

called the function: schedule_events


> {"date": ["2023-07-02"], "start_time": ["15:00"], "end_time": ["16:00"], "event": ["Masterclass with Julian"], "location": [], "description": [], "reminder": []}

In [13]:
import json

In [14]:

# Define the file path
json_file_path = 'events.json'

# Write the result to the JSON file
with open(json_file_path, 'w') as file:
    json.dump(result, file)

In [15]:
with open('events.json', 'r') as file:
    data = json.load(file)

print(data)

{"date": "2023-07-02", "event": "Masterclass with Julian"}


In [16]:
from ics import Calendar, Event

def json_to_ics(json_file_path, ics_file_path):
    # Load JSON data
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # Create a new calendar
    calendar = Calendar()

    # Iterate through each event in the JSON data
    for event_data in data['events']:
        event = Event()
        event.name = event_data.get('name', 'No Title')
        event.begin = event_data['start']
        event.end = event_data.get('end', event_data['start'])
        event.location = event_data.get('location', '')
        event.description = event_data.get('description', '')
        
        calendar.events.add(event)

    # Write the calendar to an .ics file
    with open(ics_file_path, 'w') as file:
        file.writelines(str(calendar))

# Example usage
json_file_path = 'events.json'  # Path to your JSON file
ics_file_path = 'events.ics'    # Path where the ICS file will be saved

json_to_ics(json_file_path, ics_file_path)

TypeError: string indices must be integers, not 'str'

In [ ]:
import json
from ics import Calendar, Event
from datetime import datetime, timedelta

def json_to_ics(json_file_path, ics_file_path):
    # Step 1: Read the JSON file
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    # Step 2: Create a new calendar
    calendar = Calendar()
    
    # Assuming the JSON structure is a single event
    event = Event()
    event.name = data['event']
    # Combine date and start_time for the event start
    start_datetime_str = f"{data['date']} {data['start_time']}"
    # Assuming the event duration is 1 hour for demonstration
    end_datetime = datetime.strptime(start_datetime_str, "%Y-%m-%d %H:%M") + timedelta(hours=1)
    event.begin = start_datetime_str
    event.end = end_datetime.strftime("%Y-%m-%d %H:%M")
    event.description = data.get('description', '')
    # Add the event to the calendar
    calendar.events.add(event)
    
    # Step 4: Write to an ICS file
    with open(ics_file_path, 'w') as file:
        file.writelines(str(calendar))

# Example usage
json_file_path = 'events.json'
ics_file_path = 'events.ics'
json_to_ics(json_file_path, ics_file_path)

TypeError: string indices must be integers, not 'str'